In [85]:
import helpers
import numpy as np

#helpers.npNicePrint(arr)

data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
with open("input_12.txt", 'r') as f:
    data = f.read().strip()

#grid = {(x,y): val for y, r in enumerate(data.split('\n')) for x, val in enumerate(r) if val == "#"}
gear_rows = []
for row in data.split("\n"):
    gears, config = row.split(" ")
    config = [int(x) for x in config.split(",")]
    #gear_rows.append([gears, config])
    gear_rows.append([repeat_and_join(gears, 5, "?"), config*5])

#start_coord = np.array([k for k,v in grid.items() if v == "S"][0])
gear_rows

[['??????#??#?????????#??#?????????#??#?????????#??#?????????#??#??',
  [1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1]],
 ['?#?#??##?#???#?#??##?#???#?#??##?#???#?#??##?#???#?#??##?#?',
  [2, 5, 1, 2, 5, 1, 2, 5, 1, 2, 5, 1, 2, 5, 1]],
 ['????????.?#???#??##??????????.?#???#??##??????????.?#???#??##??????????.?#???#??##??????????.?#???#??##?',
  [2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6]],
 ['#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.',
  [2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]],
 ['?##..?#?#????##..?#?#????##..?#?#????##..?#?#????##..?#?#??',
  [2, 4, 2, 4, 2, 4, 2, 4, 2, 4]],
 ['..#??##????###????#??..#??##????###????#??..#??##????###????#??..#??##????###????#??..#??##????###????#?',
  [6, 8, 6, 8, 6, 8, 6, 8, 6, 8]],
 ['???#..?#??????#..?#??????#..?#??????#..?#??????#..?#??',
  [

In [4]:
def generate_combinations(s):
    result = []

    def count_connected_hashes(s):
        counts = []
        current_count = 0

        for char in s:
            if char == '#':
                current_count += 1
            else:
                if current_count > 0:
                    counts.append(current_count)
                    current_count = 0

        if current_count > 0:
            counts.append(current_count)

        return counts

    def generate_helper(current_combination, remaining_chars):
        if not remaining_chars:
            result.append(count_connected_hashes(current_combination))
            return

        if remaining_chars[0] == '?':
            generate_helper(current_combination + '.', remaining_chars[1:])
            generate_helper(current_combination + '#', remaining_chars[1:])
        else:
            generate_helper(current_combination + remaining_chars[0], remaining_chars[1:])

    generate_helper('', s)
    return result

In [9]:
ans = 0
for gears, config in gear_rows:
    ans += sum([1 for x in generate_combinations(gears) if x == config])
ans

6488

In [11]:
def generate_combinations_with_pruning(s, row_hints):
    result = []

    def count_connected_hashes(s):
        counts = []
        current_count = 0

        for char in s:
            if char == '#':
                current_count += 1
            else:
                if current_count > 0:
                    counts.append(current_count)
                    current_count = 0

        if current_count > 0:
            counts.append(current_count)

        return counts

    def validate_combination(combination):
        # Check if the combination satisfies the row hints
        counts = count_connected_hashes(combination)
        return counts == row_hints

    def generate_helper(current_combination, remaining_chars):
        if not remaining_chars:
            if validate_combination(current_combination):
                result.append(current_combination)
            return

        if remaining_chars[0] == '?':
            generate_helper(current_combination + '.', remaining_chars[1:])
            generate_helper(current_combination + '#', remaining_chars[1:])
        else:
            generate_helper(current_combination + remaining_chars[0], remaining_chars[1:])

    generate_helper('', s)
    return result

# Example usage:
row = '???.###'
row_hints = [1, 1, 3]

combinations = generate_combinations_with_pruning(row, row_hints)
print(combinations)


['#.#.###']


In [81]:


ans = 0
for gears, config in gear_rows:
    ans += len(generate_combinations_with_pruning(gears, config))
ans

NameError: name 'generate_combinations_with_pruning' is not defined

In [5]:
gear_rows

[['???.###', [1, 1, 3]],
 ['.??..??...?##.', [1, 1, 3]],
 ['?#?#?#?#?#?#?#?', [1, 3, 1, 6]],
 ['????.#...#...', [4, 1, 1]],
 ['????.######..#####.', [1, 6, 5]],
 ['?###????????', [3, 2, 1]]]

In [17]:
def length_to_come(q):
    if len(q) == 0:
        return 0
    return(sum([x+1 for x in q]) -1)

In [95]:
import functools
# '???.###
@functools.cache
def count(gears, hint):
    if gears == "":
       return 1 if hint == () else 0

    if hint == ():
        return 0 if "#" in gears else 1
    ans = 0
    if len(gears) < length_to_come(hint):
        return 0
    
    if len(hint) == 0:
        ans += count(gears[1:], hint)
    
    if gears[0] in "?.":
        #print(gears[1:], hint)
        ans += count(gears[1:], hint)
    #print(gears, hint)
    if gears[0] in "?#" and len(gears) >= hint[0]:
        #print(gears[:hint[0]], gears[hint[0]+1])
        if all(x == "#" for x in gears[:hint[0]].replace("?", "#")):
            if not hint[0] == len(gears):
                if not gears[hint[0]] == "#":
                    #print(gears[hint[0]+1:])
                    ans += count(gears[hint[0]+1:], hint[1:])
            else:
                #print(gears[hint[0]+1:])
                ans += count(gears[hint[0]+1:], hint[1:])
            #print(gears[hint[0]+1:], hint[1:])
            
    #print(gears, hint, ans)
    return ans
#for 
count('?###????????',(3,2,1))  

10

In [51]:
"test"[3]

't'

In [9]:
all(x == "#" for x in "#?#".replace("?", "#"))

True

In [11]:
"test"[3:]

't'

In [12]:
hint = [1]
hint[1:]

[]

In [96]:
ansj = 0
for g, r in gear_rows:
    #print(g,r)
    #print(count(g,tuple(r)) )
    ansj += count(g,tuple(r))
ansj

815364548481

In [84]:
def repeat_and_join(input_string, repeat_count, separation_character):
    repeated_string = input_string * repeat_count
    joined_string = separation_character.join(repeated_string[i:i+len(input_string)] for i in range(0, len(repeated_string), len(input_string)))
    return joined_string

In [87]:
@functools.cache
def count(cfg, nums):
    if cfg == "":
        return 1 if nums == () else 0

    if nums == ():
        return 0 if "#" in cfg else 1

    result = 0
    
    if cfg[0] in ".?":
        result += count(cfg[1:], nums)
        
    if cfg[0] in "#?":
        if nums[0] <= len(cfg) and "." not in cfg[:nums[0]] and (nums[0] == len(cfg) or cfg[nums[0]] != "#"):
            result += count(cfg[nums[0] + 1:], nums[1:])

    return result

#total = 0
#
#for line in open(0):
#    cfg, nums = line.split()
#    nums = tuple(map(int, nums.split(",")))
#    total += count(cfg, nums)

#print(total)

In [ ]:
@ft.cache